Imports

In [22]:
import requests
import pandas as pd
import sqlite3
import os


Function

In [23]:
def get_temperature_data(city_name, latitude, longitude, year):
    url = (
        f"https://archive-api.open-meteo.com/v1/archive"
        f"?latitude={latitude}"
        f"&longitude={longitude}"
        f"&start_date={year}-01-01"
        f"&end_date={year}-12-31"
        f"&daily=temperature_2m_max"
        f"&timezone=Europe%2FBerlin"
    )
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        dates = data['daily']['time']
        temps = data['daily']['temperature_2m_max']
        return {
            "city": city_name,
            "year": year,
            "dates": dates,
            "temperatures": temps
        }
    else:
        print("Failed to retrieve data:", response.status_code)
        return None


Define Cities and Years

In [24]:
cities = [
    {"name": "Zurich", "lat": 47.37, "lon": 8.55},
    {"name": "Geneva", "lat": 46.20, "lon": 6.15},
    {"name": "Bern", "lat": 46.95, "lon": 7.44},
    {"name": "Berlin", "lat": 52.52, "lon": 13.41},
    {"name": "Rome", "lat": 41.89, "lon": 12.49},
    {"name": "London", "lat": 51.51, "lon": -0.13}
]
years = list(range(2014, 2024))

Collect and Aggregate Data

In [25]:
records = []

for city in cities:
    for year in years:
        result = get_temperature_data(city["name"], city["lat"], city["lon"], year)
        if result:
            avg_temp = sum(result["temperatures"]) / len(result["temperatures"])
            records.append({
                "City": result["city"],
                "Year": result["year"],
                "AvgMaxTemp": round(avg_temp, 2)
            })


SQLite

In [26]:
df = pd.DataFrame(records)
print(df.head())  # sanity check

# Save to SQLite
conn = sqlite3.connect("../climate_data.db")
df.to_sql("avg_temperatures", conn, if_exists="replace", index=False)
conn.close()


     City  Year  AvgMaxTemp
0  Zurich  2014       14.83
1  Zurich  2015       14.78
2  Zurich  2016       13.97
3  Zurich  2017       14.46
4  Zurich  2018       15.89
